### Lab:  Model Validation With Gradient Boosting

Welcome to this evening's lab!  It's going to be a fun one.  For today's class, we're going to try and take a crack at model building in a wholistic way.  

Specifically, we're going to try and do three different things:

 - Try out different versions of our data, and use our validation scores to see if something was an improvement or not
 - We're going to adjust model parameters to try and adjust our results to help curb overfitting
 - We're going to try and find model parameters that maximize our score for our dataset
 
The idea is that we'll be able to do a mini-walkthrough to test what it's like to build and validate a model and try and improve our results.

**Step 1:** Using the suggestions from the homework prompt given previously, try and add 3-4 different features ( columns ) to your data, and use your validation score to determine if they improved your results.  

This is meant to be open ended, and to allow you a chance to re-discover material from previous labs.

In [45]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.ensemble import GradientBoostingRegressor
from datetime import datetime, timedelta

def denote_null_values(df):
    """Denotes whether or not there are null values or not"""
    empty_cols_query = df.isnull().sum() > 0
    empty_df_cols = df.loc[:, empty_cols_query].columns.tolist()
    for col in empty_df_cols:
        col_name = f"{col}_missing"
        df[col_name] = pd.isnull(df[col])
    return df

def create_val_splits(df, val_units=15, return_val=False):
    """Function that will take in a dataset and split it up into training, validation, and test sets"""
    # split into training, validation, and test sets
    train = df.groupby('id').apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
    test  = df.groupby('id').apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
    if return_val:
        val   = train.groupby('id').apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
        train = train.groupby('id').apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
        return train, val, test
    else:
        return train, test



In [46]:
df = pd.read_csv('/Users/cameronlefevre/Data Science/coding/GA-DS-Class/ClassMaterial/Unit3/data/restaurants.csv', parse_dates=['visit_date'])

In [47]:
df.sort_values(by=['id', 'visit_date'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

#add new feature: month column
df['month'] = df.visit_date.dt.month

#add new feature: yesterday's visitor count
grouping = df.groupby('id').apply(lambda x: x['visitors'].shift())
df['visitors_yesterday'] = grouping.values

In [51]:
df.head(7)

,id,visit_date,visitors,calendar_date,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,month,yesterday
0,air_00a91d42b08b08d9,2016-07-01,35,2016-07-01,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN,7,NaN
1,air_00a91d42b08b08d9,2016-07-02,9,2016-07-02,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4.0,7,35.0
2,air_00a91d42b08b08d9,2016-07-04,20,2016-07-04,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN,7,9.0
3,air_00a91d42b08b08d9,2016-07-05,25,2016-07-05,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN,7,20.0
4,air_00a91d42b08b08d9,2016-07-06,29,2016-07-06,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN,7,25.0
5,air_00a91d42b08b08d9,2016-07-07,34,2016-07-07,Thursday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN,7,29.0
6,air_00a91d42b08b08d9,2016-07-08,42,2016-07-08,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN,7,34.0


In [31]:
df.loc[(df['visit_date'] == df['visit_date'][9] - timedelta(days=7)) & (df['id'] == 'air_00a91d42b08b08d9')]

,id,visit_date,visitors,calendar_date,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,month,yesterday


In [62]:
df.loc[(df['visit_date'] == '2016-07-01 00:00:00') & (df['id'] == 'air_00a91d42b08b08d9')]['visitors'][0]

35

In [ ]:
df.loc[(df['visit_date'] == '2016-07-01 00:00:00') & (df['id'] == 'air_00a91d42b08b08d9')]['visitors'][0]

**Step 2:** Try and reduce overfitting in your model, if it's persistent.  Ideally, you want your in-sample and out-of-sample scores to be about the same, or at least increasing or decreasing in proportional amounts.  

The idea here is two-fold:  see if you can narrow the gap between in-sample and out-of-sample results (using training & validation sets), while simultaneously **not** decreasing your model scores (or at least not by very much).  The idea being that the closer these two are, the more reliable your results are likely to be.

Some knobs you can turn:
 - `min_samples_leaf`: parameter in the category encoder that determines what cutoff point you can use for using the local vs. global average for the category
 - `subsample`: parameter in gbm that determines what fraction of your dataset to use at each boosting round.  This both reduces training time and makes each fitting round less related to the other
 - `max_features`: what portion of columns to use at each split.  This is very similar in purpose to `subsample`, but randomizes data at each split, vs. each round.

In [ ]:
# your code here

**Step 3:** Using the results that gave you the best answer from above, try now to find model parameters that maximize information extraction.  The three main ones are:

 - `n_estimators`:  how many boosting rounds to use
 - `learning_rate`: how much shrinkage to use at each update (keep this from .05 to .2)
 - `max_depth`: how deep each tree in your model goes
 
 **important:** fitting these things could take a looooong time.  We don't have all night.  So don't try and make this exhaustive, just try doing a little bit of parameter exploration to see if you can see in what directions to push model parameters to improve your results.  
 
 Note your validation score before proceeding to the next step.

In [ ]:
# your code here

**Step 4:** Take the best version of your model & your data, and fit it on **all** of your training + validation data.  The idea is that now that we've found the best version of what we have to work with, we want to give it as much training samples as possible.  

In [ ]:
# your code here

**Step 5:** Score your model on your test set.

Note how your validation + test scores compared to one another.

In [ ]:
# your code here